<a href="https://colab.research.google.com/github/ljkrajewski/HiDream-I1-nf4/blob/main/HiDream_4bq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/hykilpikonna/HiDream-I1-nf4.git
%cd HiDream-I1-nf4
!pip install hdi1 --no-build-isolation

Cloning into 'HiDream-I1-nf4'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 166 (delta 58), reused 52 (delta 52), pack-reused 95 (from 1)
Receiving objects: 100% (166/166), 3.07 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/HiDream-I1-nf4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 69.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.3/284.3 kB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Google proxy
LISTENING_PORT = 7860
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(" + str(LISTENING_PORT) + ")"))

In [ ]:
#!python -m hdi1 "A cat holding a sign that says 'hello world'" -m fast
!sed -i 's/demo.launch()/demo.launch(share=True)/g' /content/HiDream-I1-nf4/hdi1/web.py
!python -m hdi1.web

2025-05-09 13:28:19.775433: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-09 13:28:19.792658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746797299.813823   10444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746797299.820324   10444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-09 13:28:19.841937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr